# Simple Chat with Semantic Kernel + Ollama

This notebook demonstrates basic chat functionality using Semantic Kernel with Ollama.

**What you'll learn:**
- Semantic Kernel setup with Ollama
- Basic chat interactions using SK
- Chat history management
- Kernel functions and plugins
- Simple conversation flow with SK architecture

In [ ]:
# Import required libraries
import sys
import asyncio
sys.path.append('..')

# Semantic Kernel imports
import semantic_kernel as sk
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments

from config import config

print(f"Configuration loaded:")
print(f"  Ollama URL: {config.ollama_base_url}")
print(f"  Model: {config.model_name}")
print(f"  Temperature: {config.temperature}")

In [ ]:
# Initialize Semantic Kernel with Ollama
kernel = sk.Kernel()

# Add Ollama chat completion service
service_id = "ollama_chat"
ollama_service = OllamaChatCompletion(
    ai_model_id=config.model_name,
    host=config.ollama_base_url,
    service_id=service_id
)

kernel.add_service(ollama_service)

print("✅ Semantic Kernel with Ollama configured successfully!")
print(f"Service ID: {service_id}")
print(f"Model: {config.model_name}")
print(f"Ollama Host: {config.ollama_base_url}")

In [ ]:
# Simple chat test using Semantic Kernel
import os
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings

# Set environment variables as fallback
os.environ["OLLAMA_HOST"] = config.ollama_base_url
print(f"Set OLLAMA_HOST environment variable: {os.environ.get('OLLAMA_HOST')}")

async def simple_chat_test():
    try:
        print("Sending test message...")
        
        # Create chat history and add a message
        chat_history = ChatHistory()
        chat_history.add_user_message("Hello! Please respond with a brief greeting.")
        
        # Get the chat completion service
        chat_service = kernel.get_service(type=OllamaChatCompletion)
        
        # Debug: Check if we can inspect the service configuration
        print(f"Chat service type: {type(chat_service)}")
        
        # Create prompt execution settings
        settings = PromptExecutionSettings(
            max_tokens=500,
            temperature=config.temperature
        )
        
        # Get response
        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history,
            settings=settings
        )
        
        print(f"Response: {response[0].content}")
        print("✅ Chat is working!")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Debugging info:")
        print(f"  Config URL: {config.ollama_base_url}")
        print(f"  OLLAMA_HOST env: {os.environ.get('OLLAMA_HOST')}")
        
        # Test direct connection
        try:
            import requests
            test_url = f"{config.ollama_base_url}/api/tags"
            response = requests.get(test_url, timeout=5)
            print(f"  Direct API test: {response.status_code} - {test_url}")
        except Exception as conn_error:
            print(f"  Direct connection test failed: {conn_error}")

# Run the test
await simple_chat_test()

In [ ]:
# Create a simple chat plugin using Semantic Kernel functions
from semantic_kernel.functions import kernel_function

class SimpleChatPlugin:
    """Simple chat plugin for Semantic Kernel."""
    
    @kernel_function(
        description="Have a conversation with the assistant",
        name="chat_with_context"
    )
    async def chat_with_context(self, message: str, context: str = "") -> str:
        """Create a chat prompt with optional context."""
        
        if context:
            prompt = f"""You are a helpful assistant. 

Context: {context}

User: {message}

Please provide a helpful and relevant response."""
        else:
            prompt = f"""You are a helpful assistant.

User: {message}

Please provide a helpful response."""
        
        # Use the kernel's chat service directly
        chat_service = kernel.get_service(type=OllamaChatCompletion)
        chat_history = ChatHistory()
        chat_history.add_user_message(prompt)
        
        settings = PromptExecutionSettings(
            max_tokens=500,
            temperature=0.7
        )
        
        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history,
            settings=settings
        )
        
        return response[0].content

# Register the chat plugin
chat_plugin = SimpleChatPlugin()
kernel.add_plugin(chat_plugin, plugin_name="ChatPlugin")

print("✅ Chat plugin registered successfully!")

# Test the plugin using the correct invocation method
async def test_chat_plugin():
    try:
        # Get the function from the plugin
        chat_function = kernel.get_function("ChatPlugin", "chat_with_context")
        
        # Invoke the function
        result = await chat_function.invoke(
            kernel,
            KernelArguments(
                message="What is Python used for?",
                context="The user is learning programming"
            )
        )
        
        print(f"Plugin Response: {result.value}")
        print("✅ Plugin test successful!")
        
    except Exception as e:
        print(f"❌ Plugin error: {e}")
        
        # Alternative: Direct method call
        try:
            print("Trying direct method call...")
            result = await chat_plugin.chat_with_context(
                message="What is Python used for?",
                context="The user is learning programming"
            )
            print(f"Direct call result: {result}")
            print("✅ Direct call successful!")
        except Exception as e2:
            print(f"❌ Direct call error: {e2}")

await test_chat_plugin()

In [ ]:
# Conversation with chat history management
async def conversation_with_history():
    print("\n=== Conversation with Chat History ===")
    
    # Initialize chat history with system message
    chat_history = ChatHistory()
    chat_history.add_system_message("You are a helpful programming assistant. Keep responses concise but informative.")
    
    # Get chat service
    chat_service = kernel.get_service(type=OllamaChatCompletion)
    
    # Simulate a conversation
    conversation_turns = [
        "What is a Python list?",
        "Can you show me an example of list comprehension?",
        "How is that different from a regular for loop?"
    ]
    
    for i, user_message in enumerate(conversation_turns, 1):
        print(f"\nTurn {i}:")
        print(f"User: {user_message}")
        
        # Add user message to history
        chat_history.add_user_message(user_message)
        
        try:
            # Get response using PromptExecutionSettings
            response = await chat_service.get_chat_message_contents(
                chat_history=chat_history,
                settings=PromptExecutionSettings(
                    max_tokens=800,
                    temperature=0.7
                )
            )
            
            assistant_response = response[0].content
            print(f"Assistant: {assistant_response}")
            
            # Add assistant response to history
            chat_history.add_assistant_message(assistant_response)
            
        except Exception as e:
            print(f"Error: {e}")
            break
    
    print(f"\n✅ Conversation completed with {len(chat_history.messages)} total messages")

await conversation_with_history()

## Semantic Kernel vs LangChain Summary

### Key Differences:

**🧠 Semantic Kernel Approach:**
- **Plugin Architecture**: Organize functions into reusable plugins
- **Kernel Functions**: Use `@kernel_function` decorators for structured AI functions
- **Built-in Chat History**: Native `ChatHistory` class for conversation management
- **Service Architecture**: Clean separation between kernel, services, and plugins

**⚡ LangChain Approach:**
- **Chain Composition**: Build workflows through chain linking
- **Direct Model Invocation**: Direct `llm.invoke()` calls
- **Manual History Management**: Manual message array handling
- **Extensive Ecosystem**: More integrations and community tools

### When to Use Semantic Kernel:
- Building structured AI applications with reusable components
- Need strong plugin architecture and function organization  
- Want built-in conversation and memory management
- Prefer Microsoft's enterprise-focused approach

### Troubleshooting Network Issues:
If you encounter connection timeouts in JupyterHub/Kubernetes:

```bash
# Create External Service to bypass network policies
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: ollama-external
spec:
  type: ExternalName
  externalName: 192.168.1.81
  ports:
  - port: 11434
EOF
```

Then update config: `OLLAMA_BASE_URL=http://ollama-external:11434`